In [1]:
import pandas as pd
import numpy as np
import re
import copy
import json
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from util.api import create_lts_obj, get_pkgs_usage_count, get_packages_out_of_stackage, get_pkgs_out_transitively, get_all_time_packages, build_mtl_continuity_matrix, get_versions_df, get_update_count_df, get_count_updated_packages_by_lts, compute_monad_usage_by_df, get_added_packages_mtl_by_lts, get_removed_packages_mtl_by_lts, get_packages_started_use_mtl, get_packages_stopped_use_mtl

%matplotlib inline
plt.style.use('classic')

In [2]:
lts_list = ['0-7', '2-22', '3-22', '6-35', '7-24', '9-21', '11-22', '12-14', '12-26', '13-11', '13-19', '14-27', '15-3', '16-11']
df_list = []
df_by_category_list = []

for idx, lts in enumerate(lts_list):
    df = pd.read_pickle(f'../data/lts-{lts}/lts-{lts}.df')
    df_list.append(df)
    
for idx, lts in enumerate(lts_list):
    df = pd.read_pickle(f'../data/lts-{lts}/lts-{lts}-by-category.df')
    df_by_category_list.append(df)

In [3]:
state_monad_methods = {
    'runState': 0, 
    'evalState': 0, 
    'execState': 0, 
    'mapState': 0, 
    'withState': 0, 
    'runStateT': 0, 
    'evalStateT': 0, 
    'execStateT': 0, 
    'mapStateT': 0, 
    'withStateT': 0
}

state_monad_imports = {"Control.Monad.State", "Control.Monad.State.Class", "Control.Monad.State.Lazy", "Control.Monad.State.Strict"}

In [5]:
from operator import itemgetter

df = df_list[13]
files_info_df = df['files-info']

# recorre la informacion obtenida de cada paquete del dataframe
for files_info in files_info_df:
    # separa la informacion por cada archivo del paquete
    files_info_list = re.findall(r'(?=\{).+?(?<=\})', files_info)
    
    # recorre los archivos
    for file_info in files_info_list:
        # parsea la informacion a un diccionario
        dictionary = json.loads(file_info)
        
        # desconstruye el objeto
        (imports, calls, file) = itemgetter('imports', 'calls', 'file')(dictionary)
    
        if len(state_monad_imports.intersection(set(imports))) > 0:
            
            if len(set(state_monad_methods).intersection(set(calls))) == 0:
                print(file, calls)
                print()
                continue;
            
            
            for method in calls:
                if method in state_monad_methods:
                    state_monad_methods[method] += 1;
                    
state_monad_methods
            

/Users/feliperuiz/Desktop/github/tesis/data/lts-16-11/DAV/DAV-1.3.4/Network/Protocol/HTTP/DAV.hs ['Nothing', 'Nothing', 'DAVT', '($)', 'MonadCatch.catch', '(.)', 'runDAVT', '(.)', 'lift', '(.)', '(.)', 'lift', 'mkDAVContext', 'closeDAVContext', 'return', '($)', 'liftIO', '($)', 'liftIO', 'newManager', '($)', 'liftIO', 'parseUrl', '($)', 'return', 'return', '()', '(>>=)', 'mkDAVContext', '(>>)', '(.=)', 'basicusername', '(.=)', 'basicpassword', '(.=)', 'depth', '(.=)', 'userAgent', '(%=)', 'baseRequest', '(++)', 'catMaybes', 'mk', '(^.)', 'ctx', 'fmap', '(.)', 'mk', '(.)', 'BC8.pack', '(^.)', 'ctx', '(^.)', 'ctx', '(&&)', 'B.null', '(^.)', 'ctx', 'B.null', '(^.)', 'ctx', 'applyBasicAuth', '(^.)', 'ctx', '(^.)', 'ctx', 'return', '(=<<)', 'go', 'mkDavRequest', 'maybe', '($)', 'throwE', '(.)', 'liftIO', 'httpLbs', '(^.)', 'ctx', 'StatusCodeException', '(==)', 's', '()', 'Nothing', 'Nothing', '(.)', 'XML.renderLBS', 'davRequest', '(.=)', 'complianceClasses', '(.=)', 'allowedMethods', 'mk', 

{'runState': 31,
 'evalState': 153,
 'execState': 22,
 'mapState': 0,
 'withState': 0,
 'runStateT': 268,
 'evalStateT': 97,
 'execStateT': 32,
 'mapStateT': 26,
 'withStateT': 1}

In [12]:
cont_monad_methods = {
    'callCC': 0,
    'cont': 0, 
    'runCont': 0, 
    'mapCont': 0, 
    'withCont': 0, 
    'runContT': 0, 
    'mapContT': 0, 
    'withContT': 0, 
}

cont_monad_imports = {"Control.Monad.Cont.Class", "Control.Monad.Cont"}

In [13]:
df = df_list[13]
len(df[df["Control.Monad.Cont"] == 1])

17

In [14]:
from operator import itemgetter

df = df_list[13]
files_info_df = df['files-info']

# recorre la informacion obtenida de cada paquete del dataframe
for files_info in files_info_df:
    # separa la informacion por cada archivo del paquete
    files_info_list = re.findall(r'(?=\{).+?(?<=\})', files_info)
    
    # recorre los archivos
    for file_info in files_info_list:
        # parsea la informacion a un diccionario
        dictionary = json.loads(file_info)
        
        # desconstruye el objeto
        (imports, calls, file) = itemgetter('imports', 'calls', 'file')(dictionary)
    
        if len(cont_monad_imports.intersection(set(imports))) > 0:
            
            if len(set(cont_monad_methods).intersection(set(calls))) == 0:
                print(file, calls)
                print()
                continue;
            
            
            for method in calls:
                if method in cont_monad_methods:
                    cont_monad_methods[method] += 1;
                    
cont_monad_methods
            

/Users/feliperuiz/Desktop/github/tesis/data/lts-16-11/MonadRandom/MonadRandom-0.5.2/Control/Monad/Trans/Random/Lazy.hs ['(.)', '(.)', 'runIdentity', 'runRandT', '(.)', 'runIdentity', 'evalRandT', '(.)', 'runIdentity', 'execRandT', 'mapRandT', 'liftM', '(.)', 'LazyState.StateT', '(.)', 'LazyState.runStateT', '(.)', 'LazyState.evalStateT', '(.)', 'LazyState.execStateT', '(.)', '(.)', 'LazyState.mapStateT', '(.)', '(.)', 'LazyState.withStateT', "liftCallCC'", '(.)', 'lift', 'liftCatch', '($)', '(.)', 'randomR', '($)', '(.)', '($)', '(.)', '(.)', 'first', 'randomRs', '($)', '(.)', '(.)', 'first', '($)', '(.)', 'RandT', '($)', 'liftRandT', 'split', '($)', 'liftM', 'fst', 'LazyState.runStateT', 'lift', '(.)', 'lift', '(.)', 'lift', '(.)', 'lift', '($)', '($)', 'LazyState.liftCallCC', 'unRandT', 'f', '(.)', '($)', '($)', "LazyState.liftCallCC'", 'unRandT', 'f', '(.)', '($)', 'LazyState.liftCatch', 'unRandT', '(.)', 'unRandT', '($)', 'LazyState.liftListen', 'unRandT', '($)', 'LazyState.liftPas

{'callCC': 9,
 'cont': 0,
 'runCont': 0,
 'mapCont': 0,
 'withCont': 0,
 'runContT': 4,
 'mapContT': 0,
 'withContT': 0}